In [12]:
from langchain.prompts import PromptTemplate
from langchain.schema.messages import AIMessage
from langchain_community.llms.chatglm3 import ChatGLM3
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts.pipeline import PipelinePromptTemplate

In [13]:
endpoint_url = "http://127.0.0.1:8000/v1/chat/completions"

messages = [
    AIMessage(content="我是你的片头生成小助手"),
    AIMessage(content="我需要你为我提供你的课程名、本节课程概要"),
    AIMessage(content="以“课程名：三维建模基础；本节课程概要：本课程为三维模型高阶课程，教学围绕虚拟场景概念设计基础、场景建模工具使用和建筑类场景建模（室内场景）三个模块进行； 语气：温和”"),
]

llm = ChatGLM3(
    endpoint_url=endpoint_url,
    max_tokens=80000,
    prefix_messages=messages,
    top_p=0.9,
)

### Attaching Function Call information

In [14]:
functions = [
    {
        "name": "intro",
        "description": "A introduction for the course",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for introduction"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for introduction",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]

In [15]:
examples = [
  {
    "question": "according to the 图形学, 探索3D模型的细节增强技术，包括UV映射、纹理包装、mipmaps和纹理过滤的基本原理和应用, help me generate a introduction for this course.",
    "answer": "在我们的计算机图形学课程中，纹理映射部分将深入探讨UV映射、纹理包装、mipmaps和纹理过滤等3D模型细节增强技术的基本原理和应用。学习者将通过这些核心技术的学习，获得使数字视觉作品更加真实和具体细节的能力，为创建更加沉浸式的环境和视觉效果奠定基础。"
  },
  {
    "question": "according to the 人机交互, 深入了解视觉原则如何影响用户体验设计，包括颜色理论、布局、排版以及它们在界面设计中的应用, help me generate a introduction for this course.",
    "answer": "在我们的人机交互课程中，视觉原则部分专注于深入分析视觉设计的心理学基础，包括颜色理论、布局、排版等元素如何影响用户体验。通过本章节，学习者将掌握如何利用这些视觉原则设计出既直观又美观的用户界面，以显著提升用户在交互系统中的体验。"
  },
  {
    "question": "according to the 数据结构, 介绍栈和队列的基本概念、操作以及它们在数据管理和算法设计中的应用, help me generate a introduction for this course.",
    "answer": "我们数据结构课程中的栈与队列部分将向学生介绍这两种基本数据结构的基本概念和操作，以及它们在数据管理和算法设计中的重要应用。通过本章节的学习，学生将获得必要的知识和技能，以有效地使用栈和队列解决编程和算法中的各种问题，从而提高他们的编程能力和解决复杂问题的能力。"
  }
]


In [16]:
# template = """according to the {课程名}, {本节课程概要}, help me generate a introduction for this course."""
# prompt = PromptTemplate.from_template(template)

In [17]:
# chain = prompt | llm.bind(function_call={"name": "intro"}, functions=functions)
# chain.invoke({"课程名": "PBR三维场景建模", "本节课程概要": "本课程为三维模型高阶课程，教学围绕虚拟场景概念设计基础、场景建模工具使用和建筑类场景建模（室内场景）三个模块进行"}, config={})

ValueError: Error raised by inference endpoint: [Errno 111] Connection refused

In [ ]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import StrOutputParser

# Assuming we have examples pre-defined and we want to incorporate them into the course introduction
example_template = """Here's a detailed look at what you will learn:

Example:
Q: {example_q}
A: {example_a}"""

# Creating a single, comprehensive prompt template that focuses on the course details, including examples
full_template = """Generate a detailed course introduction for "{课程名}" which is a high-level 3D modeling course. The course focuses on virtual scene concept design, scene modeling tools, and architectural scene modeling (interior scenes). Include practical examples of what students will learn.

{example}"""

# Instantiating the PromptTemplate objects
example_prompt = PromptTemplate.from_template(example_template)
full_prompt = PromptTemplate.from_template(full_template)

# Setting up the pipeline with only the examples being dynamically generated, since that's the variable part
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, 
    pipeline_prompts=[
        ("example", example_prompt),
    ]
)

# Assuming we bind the prompt with an LLM like OpenAI's GPT and parse the structured output
chain = pipeline_prompt | llm.bind(function_call={"name": "intro"}) | StrOutputParser()

# Invoking the chain with specified course details and examples
chain.invoke({
    "课程名": "PBR三维场景建模", 
    "example_q": "如何创建逼真的室内场景？", 
    "example_a": "通过PBR材质和光照技巧，我们能够创建出既逼真又高效的室内场景。",
})


'Welcome to the "PBR三维场景建模" high-level 3D modeling course! In this course, we will explore the concepts of virtual scene design, scene modeling tools, and architectural scene modeling, specifically focusing on interior scenes. You\'ll be learning how to create highly realistic and efficient virtual environments using PBR materials and lighting techniques.\n\nThroughout the course, you will gain practical hands-on experience working with industry-standard software such as SketchUp, 3ds Max, and VRay. You will learn how to effectively use these tools to create detailed and accurate architectural scenes, including interiors, exteriors, and exoskeletons.\n\nBy the end of this course, you will have a comprehensive understanding of how to design and model virtual scenes for a variety of applications, including game development, architecture, and product visualization. Our course will provide you with the skills and knowledge needed to take your 3D modeling to the next level.\n\nSo, are you r

In [ ]:
from langchain.prompts import PromptTemplate, PipelinePromptTemplate
from langchain.schema import StrOutputParser

# 示例案例 - 输入
input_example_course_name = "三维建模基础 - 第2讲: 次世代高模制作规范（以Maya多边形建模为例）"
input_example_course_summary = "今天我们将探索如何制作高质量模型，重点是“挖洞”和“卡线”。我们会学习几种不同的“挖洞”技术，包括手动布线挤出法、布尔挖洞法、悬空法和Hardmesh运算法。每种方法都有其独特的应用场景和效果，了解这些可以帮助你在制作过程中做出更好的选择。除了“挖洞”，我们还会讲解模型的“卡线”技巧。卡线对于模型的细节保护至关重要，我们将学习如何掌握模型的卡线规范，并理解保护线的作用和原理。"

# 示例案例 - 输出
output_example = "欢迎加入“三维建模基础 - 第2讲: 次世代高模制作规范”，本节课程以Maya多边形建模为例，带你深入了解高模制作的关键技术。我们将集中探讨两大技术：挖洞与卡线，为你的模型制作提供坚实的技术支撑。..."

# 构建prompt模板，包含示例案例
full_template = f"""
以下是一个课程介绍的示例案例：

课程名：{input_example_course_name}
课程概要：{input_example_course_summary}
课程介绍：{output_example}

要求：
1. **保持专业性**：输出内容需要紧密围绕三维建模的专业知识，确保讨论的技术点精确无误。
2. **避免夸张和套话**：去除不必要的夸大和泛泛之谈，确保语言直接且实际。
3. **具体且实用**：确保讨论的内容具有实际应用价值，能够直接应用于三维建模实践中。
4. **篇幅控制**：输出内容需要控制在300-400字之间，精简而充实。
5. **逻辑清晰，条理分明**：整个课程介绍的结构需要逻辑性强，容易理解，且层次分明。
6. **总结和可能的收获**：在输出的结尾部分，提供一个总结，但不要真的写一个“总结：”，简明扼要地描述学生通过课程可能获得的知识和技能提升。
7. **语气温和**：确保输出内容的语气温和，不要过于严肃或过于随意。
8. **引导性**：确保输出内容具有引导性，能够引导学生对课程内容产生兴趣，激发学习的动力。
9. **对话客体**：确保输出内容具有能够与学生产生共鸣，让学生感觉到自己是受众之一，且符合老师的语言表达。

根据上述示例和要求，为以下课程生成一个详细的课程介绍：

课程名：{{课程名}}
课程概要：{{课程概要}}

课程介绍：
"""

# 创建PromptTemplate对象
prompt = PromptTemplate.from_template(full_template)

llm = ChatGLM3(
    endpoint_url=endpoint_url,
    max_tokens=80000,
    prefix_messages=messages,
    top_p=0.9,
)

# 结合PromptTemplate和LLM构建pipeline
chain = prompt | llm | StrOutputParser()

# 实际输入案例
real_input_course_name = "三维建模基础 - 第2讲: 次世代高模制作规范"
real_input_course_summary = "今天我们将探索如何制作高质量模型，重点是“挖洞”和“卡线”。我们会学习几种不同的“挖洞”技术，包括手动布线挤出法、布尔挖洞法、悬空法和Hardmesh运算法。每种方法都有其独特的应用场景和效果，了解这些可以帮助你在制作过程中做出更好的选择。除了“挖洞”，我们还会讲解模型的“卡线”技巧。卡线对于模型的细节保护至关重要，我们将学习如何掌握模型的卡线规范，并理解保护线的作用和原理。"

# 执行pipeline，生成课程介绍
course_introduction = chain.invoke({
    "课程名": real_input_course_name,
    "课程概要": real_input_course_summary
})



### PromptTemplate + LLM + OutputParser

In [ ]:
chain = prompt | llm | StrOutputParser()
chain.invoke({"question": "北京和上海两座城市有什么不同？"})

KeyError: "Input to PromptTemplate is missing variables {'课程概要', '课程名'}.  Expected: ['课程名', '课程概要'] Received: ['question']"

### Functions Output Parser

In [ ]:
chain = (
    prompt
    | llm.bind(function_call={"name": "introduction"}, functions=functions)
    | StrOutputParser()
)
chain.invoke({"question": "北京和上海两座城市有什么不同？"})

'北京和上海是中国两个不同的城市,它们在很多方面都有所不同。\n\n北京是中国的首都,也是历史悠久的城市之一。它有着丰富的历史文化遗产,如故宫、天安门、颐和园等,这些都是值得游览的景点。北京也是一个政治、文化和教育中心,有很多政府机构和大学。\n\n上海则是一个现代化的城市,是中国经济最发达的城市之一。它有很多高楼大厦、商业区和港口,是中国重要的国际交流中心。上海也有许多著名的旅游景点,如外滩、东方明珠塔、南京路步行街等。\n\n北京和上海的气候、饮食、文化等方面也存在一些差异。例如,北京的气候比较寒冷,冬季漫长,而上海的气候比较温和,春秋宜人。北京的饮食以烤鸭、炸酱面等为主,而上海的饮食以小笼包、生煎包等为主。\n\n北京和上海都是中国非常重要的城市,每个城市都有自己独特的魅力和特色,无论您选择哪一个城市作为旅游目的地,都会有一个难忘的旅程。'

### Simplifying input

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

map_ = RunnableParallel(question=RunnablePassthrough())
chain = (
    map_
    | prompt
    | llm.bind(function_call={"name": "introduction"}, functions=functions)
    | StrOutputParser()
)
chain.invoke("北京和上海两座城市有什么不同？")

'北京和上海是中国两个不同的城市,它们之间有许多不同之处。\n\n北京是中国的首都,有着悠久的历史和文化底蕴。它拥有许多古老的建筑和文化遗产,如紫禁城、颐和园等,也是中国的政治、文化和教育中心。北京还是中国著名的美食之都,有着丰富的中餐文化。\n\n上海则是中国最发达的城市之一,是一个现代化的国际大都市。它是中国重要的商业、金融和贸易中心,也是中国的科技和媒体 capital。上海有着许多高楼大厦和国际化的生活方式,同时也有许多传统的和历史景点,如外滩、城隍庙等。上海的餐饮文化也非常多样化,有着独特的海派菜文化。\n\n北京和上海都是中国非常重要的城市,每个城市都有自己独特的魅力和特色。'